In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# Function to load and preprocess images
def load_and_process_image(image_path, target_size=(512, 512)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = tf.keras.applications.vgg19.preprocess_input(image)
    return image

# Load content and style images
content_path = "path_to_content_image.jpg"  # Replace with your content image path
style_path = "path_to_style_image.jpg"  # Replace with your style image path

content_image = load_and_process_image(content_path)
style_image = load_and_process_image(style_path)

# Display content and style images
def display_images(content, style):
    content = np.squeeze(content, axis=0).astype("uint8")
    style = np.squeeze(style, axis=0).astype("uint8")

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(content)
    plt.title("Content Image")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(style)
    plt.title("Style Image")
    plt.axis("off")

    plt.show()

display_images(content_image, style_image)


In [ ]:
# Layers to extract content and style
content_layer = 'block5_conv2'
style_layers = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1',
    'block5_conv1'
]

# Combine all layers
all_layers = style_layers + [content_layer]


In [ ]:
# Load VGG19 model
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
vgg.trainable = False

# Create a new model to output style and content layers
outputs = [vgg.get_layer(name).output for name in all_layers]
model = tf.keras.Model([vgg.input], outputs)
print("VGG19 Model Loaded Successfully!")


In [ ]:
# Content loss
def content_loss(base_content, target_content):
    return tf.reduce_mean(tf.square(base_content - target_content))

# Style loss
def style_loss(base_style, target_style):
    base_style = tf.reshape(base_style, (-1, base_style.shape[-1]))
    target_style = tf.reshape(target_style, (-1, target_style.shape[-1]))
    base_gram = tf.matmul(base_style, base_style, transpose_a=True)
    target_gram = tf.matmul(target_style, target_style, transpose_a=True)
    return tf.reduce_mean(tf.square(base_gram - target_gram))

# Total variation loss
def total_variation_loss(image):
    return tf.image.total_variation(image)


In [ ]:
# Extract features
def get_features(image, model):
    features = model(image)
    style_features = features[:len(style_layers)]
    content_features = features[len(style_layers):]
    return style_features, content_features

content_features = get_features(content_image, model)
style_features = get_features(style_image, model)


In [ ]:
# Initialize the generated image
generated_image = tf.Variable(content_image, dtype=tf.float32)

# Optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# Style weights
style_weight = 1e-2
content_weight = 1e4

# Training loop
epochs = 1000
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        gen_style_features, gen_content_features = get_features(generated_image, model)

        # Compute style and content loss
        s_loss = tf.add_n([style_loss(gen_style_features[i], style_features[0][i]) for i in range(len(style_layers))])
        c_loss = content_loss(gen_content_features[0], content_features[1])
        t_loss = total_variation_loss(generated_image)

        total_loss = style_weight * s_loss + content_weight * c_loss + 1e-4 * t_loss

    grads = tape.gradient(total_loss, generated_image)
    optimizer.apply_gradients([(grads, generated_image)])
    generated_image.assign(tf.clip_by_value(generated_image, 0, 255))

    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {total_loss.numpy()}")

print("Style Transfer Complete!")


In [ ]:
# Convert image back to valid range and display
final_image = generated_image.numpy()
final_image = np.squeeze(final_image, axis=0)
final_image = np.clip(final_image, 0, 255).astype('uint8')

plt.figure(figsize=(8, 8))
plt.imshow(final_image)
plt.title("Generated Image")
plt.axis("off")
plt.show()
